In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import skimage as sk
from skimage import transform
import random
from PIL import Image
import os
import matplotlib.pyplot as plt

from utils import train_test_split


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
np.mean([1.2])

1.2

In [3]:
def rotate_img(image):
    random_degree = random.uniform(-25, 25) #25% from left or right
    return sk.transform.rotate(image, random_degree)

def noise_img(image):
    return sk.util.random_noise(image)

In [4]:
with tf.Session() as sess:
    devices = sess.list_devices()
    for d in devices:
        print(d)

_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 14470206343894414656)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11171775893897263048)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 5921705883225204132)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 7518981325, 16085904074211201974)


In [5]:
flooded_img = []
nonflooded_img = []
unlabeled_img = []

h_dim  = 1000
v_dim = 750

root = "Train"
flood_dir = os.path.join(root,'Labeled/Flooded/image/')
nonflood_dir = os.path.join(root,'Labeled/Non-Flooded/image/')
unlabel_dir = os.path.join(root,'Unlabeled/image/')

for file in os.listdir(flood_dir):
    image = Image.open(os.path.join(flood_dir, file))
    image = np.array(image.resize((h_dim,v_dim)))
    flooded_img.append(rotate_img(image))

for file in os.listdir(nonflood_dir):
    image = Image.open(os.path.join(nonflood_dir, file))
    image = np.array(image.resize((h_dim,v_dim)))
    nonflooded_img.append(rotate_img(image))
    
for file in os.listdir(unlabel_dir):
    image = Image.open(os.path.join(unlabel_dir, file))
    image = np.array(image.resize((h_dim,v_dim)))
    unlabeled_img.append(rotate_img(image))
    unlabeled_img.append(rotate_img(image))

In [6]:
print("Flooded Image Shape: {}".format(flooded_img[0].shape))
print("Non_Flooded Image Shape: {}".format(nonflooded_img[0].shape))
print("Unlabeled Image Shape: {}".format(unlabeled_img[0].shape))

Flooded Image Shape: (750, 1000, 3)
Non_Flooded Image Shape: (750, 1000, 3)
Unlabeled Image Shape: (750, 1000, 3)


In [7]:
data_img = np.vstack((np.array(flooded_img), np.array(nonflooded_img))) / 255.
data_img.shape

(397, 750, 1000, 3)

In [8]:
unlabeled_img = np.array(unlabeled_img) /255.
unlabeled_img.shape

(366, 750, 1000, 3)

In [9]:
#train_idx = np.array([np.arange(7),np.arange(10,17)]).flatten()
#test_idx = np.array([np.arange(7,10),np.arange(17,20)]).flatten()

test = False

#n is number images from each class (flooded or non flooded)
if test == True:
    n = 20
else:
    n = min(len(flooded_img),len(nonflooded_img))


idxs = train_test_split(n,flooded_img,nonflooded_img,unlabeled_img)
label_train_idx, label_test_idx, train_labels, test_labels, unlabel_train_idx, unlabel_test_idx = idxs

Training Index: [  0   1   2   3   4   5   6   8   9  10  11  12  15  16  17  19  22  23
  25  26  27  29  30  31  32  33  34  35  36  38  39  40  41  42  43  45
  47  48  49  50  51  56  71  73  77  79  83  89 102 104 114 115 120 133
 134 150 154 155 159 174 221 232 240 251 265 281 283 297 301 303 306 309
 312 358 362 365 366 367 374 394]
Testing Index: [  7  13  14  18  20  21  24  28  37  44  46  99 109 148 162 202 205 217
 226 230 259 347]
Unlabeled Training Index: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 20 23 24 25 27 29 30
 32 33 35 36 37 38 39 40 42 44 45 46 47 48 49 50]


### CNN CODE

In [10]:
# convolutional layer
def conv_layer(x, shape):

    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    bias = tf.Variable(tf.constant(0.05, shape=[shape[-1]]))

    out = tf.nn.conv2d(input=x, filters=weights, strides=[1,1,1,1], padding='SAME')
    out += bias
    return out

# pooling layer
def max_pool(x, k=2):

    out = tf.nn.max_pool(value=x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
    return out

# fully connected layer
def fully_connected_layer(x, shape):

    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    bias = tf.Variable(tf.constant(0.05, shape=[shape[1]]))

    out = tf.matmul(a=x, b=weights)
    out += bias
    return out

# flatten layer
def flatten_layer(x):
    
    size = x.get_shape()[1:4].num_elements()
    out = tf.reshape(x, [-1,size])
    return out, size

# relu
relu = lambda x: tf.nn.relu(features=x)

# softmax
softmax = lambda x: tf.nn.softmax(logits=x)

In [11]:
#shape = [filter_size, filter_size, num_input_channels, num_filters]

# define CNN
def toy_model(x):

    # three convolutional layers with max pool
    shape0 = [5, 5, 3, 3]
    conv0 = conv_layer(x, shape0)
    conv0 = max_pool(conv0, k=2)

    shape1 = [5, 5, 3, 1]
    conv1 = conv_layer(conv0, shape1)
    conv1 = max_pool(conv1, k=2)

    shape2 = [5, 5, 1, 1]
    conv2 = conv_layer(conv1, shape2)
    conv2 = max_pool(conv2, k=2)

    # flatten output and put through a fully connected layer
    flat1, size1 = flatten_layer(conv2)
    fc1 = fully_connected_layer(flat1, [size1, 64])
    fc1 = relu(fc1)

    fc2 = fully_connected_layer(fc1, [64, 1])
    out = softmax(fc2)

    return out

In [12]:
def generate_guess_label(pred_u_raw, k):
    # guess label = average prediction over k augmentations of same image
    # num_images = pred_u_raw.shape[0].value / k # Throws error of NoneType and int since pred_u_raw.shape[0].value is None
    
    try:
        num_images = int(pred_u_raw.shape[0].value / k)

        idx = 0
        temp_labels = []
        for i in range(num_images):
            temp_labels.append(tf.reduce_mean(pred_u_raw[idx:idx+k]))
            idx += k

        # repeat label for each augmentation
        guess_labels = tf.repeat(tf.stack(temp_labels), k)

        # reshape and remove gradient tracking
        guess_labels = tf.reshape(guess_labels, (-1,1))
        guess_labels = tf.stop_gradient(guess_labels)

        return guess_labels

    except TypeError:
      
      return pred_u_raw

In [13]:
# define inputs
hdim = data_img[0].shape[0]
vdim = data_img[0].shape[1]
x = tf.placeholder(tf.float32, [None, hdim, vdim, 3]) # labeled images (augmented)
u = tf.placeholder(tf.float32, [None, hdim, vdim, 3]) # unlabeled images (augmented)
y = tf.placeholder(tf.float32, [None, 1]) # labels
y_train_true = np.array(train_labels).reshape(-1,1)
y_test_true = np.array(test_labels).reshape(-1,1)
k = 2 # augment images k times

# Google paper section 3.5 says 100 is a good place to start for w_unlabeled
# Google paper also suggests ramping up value to 100 over first 16,000 epochs
w_unlabeled = 100. 

# run model with placeholder tensors
pred_x = toy_model(x)
pred_u_raw = toy_model(u)

# calculate guess labels for unlabeled images 
pred_u = generate_guess_label(pred_u_raw, k)

# sharpen guess labels here? is this necessary for only 2 classes?

# define loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=pred_x, labels=y)
labeled_loss = tf.reduce_mean(cross_entropy)
unlabeled_loss = tf.nn.l2_loss(pred_u - pred_u_raw)
cost = labeled_loss + w_unlabeled*unlabeled_loss

# define accuracy
pred_correct = tf.equal(tf.argmax(pred_x, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

# initialize variables
init = tf.global_variables_initializer()
training_iters = 3
batch_size = 4 #len(train_idx)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## Train Model

In [ ]:
# train model
with tf.Session() as sess:
    sess.run(init)
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    num_batches = len(label_train_idx)//batch_size

    
    for i in range(training_iters):
        
        # Reset metrics
        loss_total = 0
        acc_total = 0
   
        # Run optimization 
        # Calculate batch loss and accuracy
        for batch in range(num_batches):
            batch_x = data_img[label_train_idx,:,:,:][batch*batch_size:min((batch+1)*batch_size, len(label_train_idx))]
            batch_u = unlabeled_img[unlabel_train_idx,:,:,:][batch*k*batch_size:min((batch+1)*k*batch_size, len(unlabel_train_idx))]
            batch_y = train_labels[batch*batch_size:min((batch+1)*batch_size, len(train_labels))]   

            feed_dict={x: batch_x, u: batch_u, y: batch_y}
            opt = sess.run(optimizer, feed_dict=feed_dict)
            loss, acc = sess.run([cost, accuracy], feed_dict=feed_dict)
            loss_total += loss
            acc_total += acc

        # Average metrics
        ave_loss = loss_total/num_batches
        ave_acc = acc_total/num_batches

        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(ave_loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(ave_acc))

        # Calculate accuracy for all test images
        test_acc,valid_loss = sess.run([accuracy,cost],
                                feed_dict={x: data_img[label_test_idx,:,:,:], u: unlabeled_img[unlabel_test_idx,:,:,:], y : test_labels})
        train_loss.append(ave_loss)
        test_loss.append(valid_loss)
        train_accuracy.append(ave_acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()